# ADME PREDCT FUNCTION

In [12]:
import pandas as pd
from matplotlib import pyplot as plt
from sklearn import metrics
from loguru import logger
import os
import time
import datetime
DATASET_PATH="../dataset"
LOG_PATH="../logs/"
MODEL_PATH="../models/"
RESULT_PATH="../result/"
CHEMPROP_RESULT="../models/"
import sys
sys.path.append("../chemprop")
sys.path.append("../qsprpred")
sys.path.append("../")
import chemprop
#Save and load models
import joblib
import pickle
#Metrics 
import sklearn.metrics as metrics
from sklearn.feature_selection import mutual_info_regression

pd.set_option('max_colwidth',200)
pd.set_option('display.width',200)
pd.set_option('display.max_columns',200)
import time
from loguru import logger
import datetime
import os,sys
import time
import pandas as pd
from pathlib import Path
project_path = Path.cwd().parent
log_path = Path(project_path, "log")
log_time = time.strftime("%Y_%m")
log_path="./logs"
if ~os.path.exists(log_path):
    os.makedirs(log_path, exist_ok=True)
logger.remove()
logger.add(f'logs/nca2_{log_time}.log', format="{time:YYYY-MM-DD HH:mm:ss} | "
                               "{file} | "
                               "{process.name} | "
                               "{thread.name} | "
                               "{module}.{function}:{line} - {level} -{message}",level="INFO", enqueue=True,encoding="utf-8", rotation="12:00")
logger.add(sys.stderr, format='{time:YYYY-MM-DD HH:mm:ss} | ' 
                                "{file} | "# 
                               "{process.name} | "  # 进程名
                               "{thread.name} | "  # 进程名
                               '{module}.{function}:{line} - {level} -{message}',level="INFO", enqueue=True
)
DEBUG=True
def get_cost_time(function):
    """
    计算程序运行时间
    :param function:
    :return:
    """
    def func(*args, **kwargs):
        if DEBUG:
            start_time = datetime.datetime.now()
            result = function(*args, **kwargs)
            end_time = datetime.datetime.now()
            seconds = (end_time - start_time).seconds
            start = start_time.strftime('%Y-%m-%d %H:%M')
            minutes = seconds // 60
            second = seconds % 60
            timeStr = str(minutes) + '分钟' + str(second) + "秒"
            logger.info('---function: %s，cost time:%s seconds---' %( function.__name__,timeStr))
        else:
            result = function(*args, **kwargs)
        return result
    return func
class StopWatch:

    def __init__(self):
        self.start = datetime.datetime.now()

    def reset(self):
        self.start =  datetime.datetime.now()

    def stop(self, msg='Time it took: '):
        time_range =  datetime.datetime.now() - self.start
        seconds = time_range.seconds
        start_time = self.start.strftime('%Y-%m-%d %H:%M')
        logger.info('---%s start from %s, total cost time:%s seconds---' %(msg,start_time,seconds))
        self.reset()
        return seconds
now_time = datetime.datetime.now()
data_str = now_time.strftime('%Y_%m_%d_%H_%M')
import os
from typing import List
def get_feature_param(data_name):
    config_json=CHEMPROP_RESULT+"%s/args.json"%data_name
    feature_param=""
    import json
    logger.info("json:%s"%config_json)
    with open(config_json, 'r') as f:
        json_config = json.loads(f.read())
        print(json_config['features_generator'])
        if not json_config['features_generator'] is None:
            print(json_config)
            print(json_config['features_generator'][0])
            feature_param=json_config['features_generator'][0]
        else:
            print(json_config['features_generator'] is None)
    return feature_param
def read_file(file_path):
    # 获取文件扩展名
    file_extension = os.path.splitext(file_path)[1]

    # 读取 Excel 文件
    if file_extension == '.xlsx' or file_extension == '.xls':
        df = pd.read_excel(file_path, engine='openpyxl')

    # 读取 CSV 文件
    elif file_extension == '.csv':
        df = pd.read_csv(file_path)

    else:
        raise ValueError(f"Unsupported file type: {file_extension}")

    return df
def predict(data_name,smiles_column,test_file,feature_param=None):
    arguments = [
        '--test_path', test_file,
        '--preds_path', 'preds/test_preds_%s_%s.csv'%(data_name,data_str),
        '--smiles_column',smiles_column,
        # '--gpu','1', # if gpu set it 
        '--checkpoint_dir', CHEMPROP_RESULT+data_name
    ]
    if feature_param=="morgan":
        arguments.extend(['--features_generator','morgan'])
    elif feature_param=="rdkit_2d_normalized":
        arguments.extend([ '--no_features_scaling', '--features_generator','rdkit_2d_normalized'])

    # logger.info(arguments)
    args = chemprop.args.PredictArgs().parse_args(arguments)
    preds = chemprop.train.make_predictions(args=args)

    df = pd.read_csv(test_file)
    print(test_file)
    df['preds'] = [x[0] for x in preds]

    logger.info(df)
    # df.reset_index(inplace=True)
    # df = df.drop(columns=df.filter(regex='Unnamed').columns)
   
 
    return df
def get_predict_result(test_data_path,smiles_column,data_name,feature_param):
# fup_log10fup_log10_pred_result_2023_06_08_20_20.csv

    result =predict(data_name,smiles_column,test_data_path,feature_param)

    print(result)
    return result 

def predict_pk(test_data_path,exp_name,data_name,smiles_column="SMILES"):
    feature_param = get_feature_param(data_name)
    df_result = get_predict_result(
        test_data_path, smiles_column, data_name, feature_param
    )
   
    print(test_data_path)
    if exp_name!="caco2":
        df_result[exp_name+"_preds"] = df_result["preds"].apply(lambda x: (10**x))
    elif exp_name!="cl":
        df_result[exp_name+"_preds"] = df_result["preds"].apply(lambda x: (10**x)/ 1000 * 60) 
    else:
        df_result[exp_name+"_preds"] = df_result["preds"].apply(lambda x: (10**6) * (10**x))
    df_result=df_result.drop(columns=["preds"])
  
    return df_result


# RUN ADME PREDICT

In [11]:
predict_data_file= "../dataset/PK_molecule_smiles.xlsx" ## 修改输入文件
df_test_data = pd.read_excel(predict_data_file, engine='openpyxl')
logger.info(df_test_data.shape)
logger.info(df_test_data.columns)
print(df_test_data.head(10))
predict_data_file=predict_data_file.replace(".xlsx",".csv")
df_test_data.to_csv(predict_data_file,index=False)
# predict_data_file
best_model={  
    # "vdss":"vdss_log10_32_200_rdkit_2d_normalized_2023_06_27_15_49",
    "cl":"cl_1215_log10_32__2023_07_08_22_52",
     "fu":"fu_log10_32_ori__2023_06_20_15_46",
    "caco2":"caco2_log10_2023_06_10_22_01"
}
df_result=pd.DataFrame()
df_result_pk=pd.DataFrame()
for exp_name, model_name in best_model.items():
    print(f"======target : {exp_name}, best model: {model_name}===========")
    df_result_pk=predict_pk(predict_data_file,exp_name,model_name)
    # df_result_pk=df_result_pk.drop(columns=['level_0','index'])
    logger.info(df_result_pk.head(1))
    if df_result.empty:
        df_result=df_result_pk
    else:
        df_result = df_result.merge(df_result_pk, on= [ 'Drug', 'SMILES'])
    logger.info(df_result.head(5))


print(df_result.columns)
print(df_result.head(2))
now_time = datetime.datetime.now()
data_str = now_time.strftime('%Y_%m_%d_%H_%M')
predict_result_file="../result/df_pred_result.csv"
df_result.to_csv(predict_result_file,index=False)

2024-01-06 22:37:08 | 3006981774.py | MainProcess | MainThread | 3006981774.<module>:3 - INFO -(139, 2)
2024-01-06 22:37:08 | 3006981774.py | MainProcess | MainThread | 3006981774.<module>:4 - INFO -Index(['Drug', 'SMILES'], dtype='object')
2024-01-06 22:37:08 | 797016409.py | MainProcess | MainThread | 797016409.get_feature_param:36 - INFO -json:../models/cl_1215_log10_32__2023_07_08_22_52/args.json


            Drug                                                                 SMILES
0      Abanoquil                          COc1cc2CCN(Cc2cc1OC)c3cc(N)c4cc(OC)c(OC)cc4n3
1     Adinazolam                              CN(C)Cc1nnc2CN=C(c3ccccc3)c4cc(Cl)ccc4n12
2       AFN-1252                      Cc1c2ccccc2oc1CN(C)C(=O)/C=C/c3cc4c(nc3)NC(=O)CC4
3   AG-EE-388 ZW         C(c1c(cc(cc1)CC(=O)N[C@@H](CC(C)C)c1c(cccc1)N1CCCCC1)OCC)(=O)O
4       Ajmaline  CC[C@@H]1[C@H](N2[C@H]3CC1C4[C@@H]2CC5([C@H]3N(c6c5cccc6)C)[C@@H]4O)O
5      Alectinib      CCc1cc2c(cc1N3CCC(CC3)N4CCOCC4)C(c5c(c6ccc(cc6[nH]5)C#N)C2=O)(C)C
6     Alprenolol                                              CC(C)NCC(O)COc1ccccc1CC=C
7  Amitriptyline                                         CN(C)CCC=C1c2ccccc2CCc3ccccc13
8      Amonafide                                c12c3C(N(CCN(C)C)C(c1cccc2cc(c3)N)=O)=O
9        Amsalog               CNC(=O)c1cccc2c(Nc3ccc(NS(=O)(=O)C)cc3OC)c4cccc(C)c4nc12
======target : cl, best model: c

139it [00:00, 120880.83it/s]
100%|██████████| 139/139 [00:00<00:00, 135520.28it/s]

Validating SMILES


Test size = 139


...

Loading pretrained parameter "encoder.encoder.0.cached_zero_vector".
Loading pretrained parameter "encoder.encoder.0.W_i.weight".
Loading pretrained parameter "encoder.encoder.0.W_h.weight".
Loading pretrained parameter "encoder.encoder.0.W_o.weight".
Loading pretrained parameter "encoder.encoder.0.W_o.bias".
Loading pretrained parameter "readout.1.weight".
Loading pretrained parameter "readout.1.bias".
Loading pretrained parameter "readout.4.weight".
Loading pretrained parameter "readout.4.bias".


 80%|████████  | 4/5 [01:19<00:19, 19.97s/it]

Loading pretrained parameter "encoder.encoder.0.cached_zero_vector".
Loading pretrained parameter "encoder.encoder.0.W_i.weight".
Loading pretrained parameter "encoder.encoder.0.W_h.weight".
Loading pretrained parameter "encoder.encoder.0.W_o.weight".
Loading pretrained parameter "encoder.encoder.0.W_o.bias".
Loading pretrained parameter "readout.1.weight".
Loading pretrained parameter "readout.1.bias".
Loading pretrained parameter "readout.4.weight".
Loading pretrained parameter "readout.4.bias".


100%|██████████| 5/5 [01:40<00:00, 20.05s/it]
2024-01-06 22:42:13 | 797016409.py | MainProcess | MainThread | 797016409.predict:84 - INFO -             Drug                                                                                          SMILES     preds
0       Abanoquil                                                   COc1cc2CCN(Cc2cc1OC)c3cc(N)c4cc(OC)c(OC)cc4n3 -5.166030
1      Adinazolam                                                       CN(C)Cc1nnc2CN=C(c3ccccc3)c4cc(Cl)ccc4n12 -4.784298
2        AFN-1252                                               Cc1c2ccccc2oc1CN(C)C(=O)/C=C/c3cc4c(nc3)NC(=O)CC4 -4.523219
3    AG-EE-388 ZW                                  C(c1c(cc(cc1)CC(=O)N[C@@H](CC(C)C)c1c(cccc1)N1CCCCC1)OCC)(=O)O -5.322564
4        Ajmaline                           CC[C@@H]1[C@H](N2[C@H]3CC1C4[C@@H]2CC5([C@H]3N(c6c5cccc6)C)[C@@H]4O)O -4.955543
..            ...                                                                                             ...    

Saving predictions to preds/test_preds_caco2_log10_2023_06_10_22_01_2024_01_06_22_37.csv
Elapsed time = 0:01:41
../dataset/PK_molecule_smiles.csv
             Drug                                                                                          SMILES     preds
0       Abanoquil                                                   COc1cc2CCN(Cc2cc1OC)c3cc(N)c4cc(OC)c(OC)cc4n3 -5.166030
1      Adinazolam                                                       CN(C)Cc1nnc2CN=C(c3ccccc3)c4cc(Cl)ccc4n12 -4.784298
2        AFN-1252                                               Cc1c2ccccc2oc1CN(C)C(=O)/C=C/c3cc4c(nc3)NC(=O)CC4 -4.523219
3    AG-EE-388 ZW                                  C(c1c(cc(cc1)CC(=O)N[C@@H](CC(C)C)c1c(cccc1)N1CCCCC1)OCC)(=O)O -5.322564
4        Ajmaline                           CC[C@@H]1[C@H](N2[C@H]3CC1C4[C@@H]2CC5([C@H]3N(c6c5cccc6)C)[C@@H]4O)O -4.955543
..            ...                                                                                             